#### Collaborative Filtering Technique
* Nearest Neighbours Model - use ratings of most similar user

In [119]:
import pandas as pd
dataFile='BX-CSV-Dump/BX-Book-Ratings.csv'
data = pd.read_csv(dataFile, sep=";", header=0, names=["user", "isbn", "rating"], encoding = "ISO-8859-1")

In [120]:
data.head()

,user,isbn,rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [121]:
bookFile='BX-CSV-Dump/BX-Books.csv'
books = pd.read_csv(bookFile, sep=";", header=0, error_bad_lines=False, usecols=[0, 1, 2], index_col=0, names=["isbn", "title", "author"], encoding = "ISO-8859-1")

In [122]:
books.head()

,title,author
isbn,,
0195153448,Classical Mythology,Mark P. O. Morford
0002005018,Clara Callan,Richard Bruce Wright
0060973129,Decision in Normandy,Carlo D'Este
0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata
0393045218,The Mummies of Urumchi,E. J. W. Barber


In [123]:
def bookMeta(isbn):
    title = books.at[isbn,"title"]
    author = books.at[isbn,"author"]
    return title, author
bookMeta("0671027360")

('Angels &amp; Demons', 'Dan Brown')

In [124]:
data = data[data["isbn"].isin(books.index)]

In [125]:
def faveBooks(user,N):
    userRatings = data[data["user"]==user]
    sortedRatings = pd.DataFrame.sort_values(userRatings,['rating'],ascending=[0])[:N] 
    sortedRatings["title"] = sortedRatings["isbn"].apply(bookMeta)
    return sortedRatings

In [126]:
faveBooks(204622,5)

,user,isbn,rating,title
844955,204622,0967560500,10,"(Natural Hormonal Enhancement, Rob Faigin)"
844935,204622,0671027360,10,"(Angels &amp; Demons, Dan Brown)"
844926,204622,0385504209,10,"(The Da Vinci Code, Dan Brown)"
844958,204622,097173660X,9,"(Life After School Explained, Cap &amp; Compass)"
844920,204622,0060935464,9,"(To Kill a Mockingbird, Harper Lee)"


In [127]:
data.shape

(1031175, 3)

In [128]:
usersPerISBN = data.isbn.value_counts()
usersPerISBN.head(10)

0971880107    2502
0316666343    1295
0385504209     883
0060928336     732
0312195516     723
044023722X     647
0142001740     615
067976402X     614
0671027360     586
0446672211     585
Name: isbn, dtype: int64

In [129]:
usersPerISBN.shape

(270170,)

In [130]:
ISBNsPerUser = data.user.value_counts()

In [131]:
ISBNsPerUser.shape

(92107,)

In [132]:
data = data[data["isbn"].isin(usersPerISBN[usersPerISBN>10].index)]

In [133]:
data = data[data["user"].isin(ISBNsPerUser[ISBNsPerUser>10].index)]

In [134]:
data.shape

(405709, 3)

In [135]:
from scipy.sparse import coo_matrix
data['user'] = data['user'].astype("category")
data['isbn'] = data['isbn'].astype("category")

R = coo_matrix((data['rating'].astype(float),
                       (data['user'].cat.codes.copy(),
                        data['isbn'].cat.codes.copy())))

In [113]:
R

<10706x15451 sparse matrix of type '<class 'numpy.float64'>'
	with 405709 stored elements in COOrdinate format>

In [114]:
R.shape

(10706, 15451)

In [38]:
len(R.data)

405709

In [50]:
R.row[0], R.row[1], R.row[2], R.row[88434]

(10633, 10633, 10633, 2208)

In [43]:
R.col[0], R.col[2], R.col[1]

(3053, 7873, 4025)

In [68]:
R.data[90]

9.0

In [77]:
len(R.row)

405709

In [72]:
R.row[0]

10633

In [73]:
ratings = R.data

### Intialize factor matrices

In [136]:
M, N = R.shape
K = 3 #Factors

In [137]:
import numpy as np
P = np.random.rand(M, K)
Q = np.random.rand(K, M)

### Compute the error

In [84]:
# from numpy.linalg import norm

# def error(R, P, Q, lamda=0.02):
#     ratings = R.data #Accessing non-null ratings 
#     rows = R.row #Corresponing row id 
#     cols = R.col #Corresponing column id
#     e = 0 #Intialize error zero
#     #Iterate through all the ratings that are already known and compute individual error
#     for ui in range(len(ratings)): 
#         rui = ratings[ui]
#         u = rows[ui]
#         i = cols[ui]
#         #If rating is greater than 0 then apply optimization algo
#         if rui > 0:
#             e = e + pow(rui-np.dot(P[u,:], Q[:,i]), 2) + \
#                 lamda*(pow(norm(P[u,:]), 2) + pow(norm(Q[i,:]), 2))
#     return e

In [138]:
from numpy.linalg import norm

def error(R,P,Q,lamda=0.02):
    ratings = R.data
    rows = R.row
    cols = R.col
    e = 0 
    for ui in range(len(ratings)):
        rui=ratings[ui]
        u = rows[ui]
        i = cols[ui]
        if rui>0:
            e= e + pow(rui-np.dot(P[u,:],Q[:,i]),2)+\
                lamda*(pow(norm(P[u,:]),2)+pow(norm(Q[:,i]),2))
    return e

In [139]:
error(R,P,Q)

IndexError: index 15311 is out of bounds for axis 1 with size 10706